In [1]:
# твой список классов
classes = [
    "healthy tomato",
    "tomato early blight",
    "tomato late blight",
    "powdery mildew"
]

classes

['healthy tomato',
 'tomato early blight',
 'tomato late blight',
 'powdery mildew']

In [16]:
import requests
import json
import base64
from pathlib import Path

# путь к изображению
image_path = Path(r"apple_scab_1465_180.jpg")

# классы, между которыми нужно выбрать
classes = ['Tomato', 'leaf blight', 'Raspberry', 'Scab Leaf', 'grape', 'Soyabean', 
           'leaf black rot', 'Bell_pepper', 'mold leaf', 'Early blight leaf', 'Powdery mildew leaf', 
           'leaf bacterial spot', 'rust leaf', 'leaf early blight', 'Corn', 'Potato', 'Blueberry', 'Apple', 
           'leaf mosaic virus', 'leaf yellow virus', 'Squash', 'Strawberry', 'Septoria leaf spot', 'Gray leaf spot', 
           'Cherry', 'leaf spot', 'Peach', 'leaf late blight','Healthy']

class OllamaVisionAnalyzer:
    def __init__(self, model="qwen2.5vl:3b", host="http://localhost:11434"):
        self.model = model
        self.host = host
    
    def encode_image(self, image_path):
        """Convert image to base64 encoding"""
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    def analyze_image(self, image_path, 
             prompt=f"""Determine the plant class and the plant disease class shown in the image.
                        Possible classes: {', '.join(classes)}.
                        If the confidence is below 70%, return the class "undefined".

                        Answer in the following format:

                        Plant class: <name>
                        Disease class: <name>
                        Plant confidence: <percentage from 0 to 100>
                        Disease confidence: <percentage from 0 to 100>
                        Explanation: <why these classes were chosen>
                        """
):
        """Send image to Ollama for analysis"""
        
        # Encode the image
        image_data = self.encode_image(image_path)
        
        # Prepare the request
        payload = {
            "model": self.model,
            "prompt": prompt,
            "images": [image_data],
            "stream": False
        }
        
        # Make the API call
        response = requests.post(
            f"{self.host}/api/generate",
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            return response.json()["response"]
        else:
            raise Exception(f"API Error: {response.status_code}")

# Usage example
analyzer = OllamaVisionAnalyzer()
description = analyzer.analyze_image(image_path)
print(description)

Plant class: Tomato
Disease class: Early blight leaf
Plant confidence: 80%
Disease confidence: 90%
Explanation: The image shows a leaf with brown spots and discoloration, which are characteristic of early blight, a common fungal disease affecting tomatoes. The high confidence is based on the visible symptoms and the known association between these symptoms and early blight.


In [4]:
import os
import pandas as pd
import numpy as np

In [13]:
base_dir = 'datasets/PlantDoc-Dataset-windows-compatible/train'

plant_labels = []
for imagePath in os.listdir(base_dir):
	l = label = imagePath.split(os.path.sep)[-1].split(" ",1)
	plant_labels.append(l)

plant_list = []
disease_list = []
for item in plant_labels:
    if item[0] not in classes_list:
        classes_list.append(item[0])
    if item[1] not in classes_list:
        classes_list.append(item[1])

for lists in os.listdir(base_dir):
  path = os.path.join(base_dir, lists)
  for filename in glob.glob(path+'/*'):
    p_l = lists.split(os.path.sep)[-1].split(" ")
    img_class=[]
    if p_l[0] in classes_list:
      img_class.append(p_l[0])
    if p_l[1] in classes_list:
      img_class.append(p_l[1])

In [14]:
classes_list

['Apple',
 'Healthy',
 'Grape',
 'Scab',
 'Tomato',
 'Mosaic virus',
 'Soyabean',
 'Corn',
 'Gray leaf spot',
 'Blueberry',
 'Healty',
 'Potato',
 'Early blight',
 'Peach',
 'Rust',
 'Strawberry',
 'Yellow leaf virus',
 'Twospotted spider mite',
 'Late blight',
 'Black rot',
 'Bell_pepper',
 'Raspberry',
 'Bacterial spot',
 'Cherry',
 'mold leaf',
 'Squash',
 'Powdery mildew',
 'leaf blight',
 'Septoria leaf spot']

In [ ]:
with open('plantdoc_annotation.csv', 'w', newline='') as file:
  file.write("id;classes;")
  file.write('\n')
  for lists in os.listdir(base_dir):
    path = os.path.join(base_dir, lists)
    for filename in glob.glob(path+'/*'):
      file.write(os.path.basename(filename))
      file.write(';')
      p_l = lists.split(os.path.sep)[-1].split("___")
      img_class=[]
      if p_l[0] in classes_list:
        img_class.append(p_l[0])
      if p_l[1] in classes_list:
        img_class.append(p_l[1])
      file.write(str(img_class))
      file.write(';')
      file.write('\n')

In [ ]:
import os
import time
import base64
import csv
import requests

# -------- CONFIG --------
DATASET_DIR = Path(r"datasets\PlantDoc-Dataset-windows-compatible\train")   # папка с изображениями
OUTPUT_CSV = "classification_results.csv"
MODEL_NAME = "qwen2.5vl:3b"

# твой список классов
classes = [
    "healthy tomato",
    "tomato early blight",
    "tomato late blight",
    "powdery mildew"
]

# ---- JSON-ориентированный промпт ----
PROMPT_TEMPLATE = f"""
Determine the plant class and the plant disease class shown in the image.
Possible classes: {', '.join(classes)}.
If the model’s confidence for any category is below 70%, return the class "undefined".

Respond ONLY in valid JSON with the following structure:

{{
  "plant_class": "<name or 'undefined'>",
  "disease_class": "<name or 'undefined'>",
  "plant_confidence": <number from 0 to 100>,
  "disease_confidence": <number from 0 to 100>,
  "explanation": "<short explanation>"
}}
"""

# --------------------------------------


def encode_image(path):
    """Читает изображение и возвращает base64."""
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def classify_image(image_b64):
    """Отправляет запрос в Ollama и возвращает JSON-ответ модели."""
    data = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "user",
                "content": PROMPT_TEMPLATE,
                "images": [image_b64]
            }
        ]
    }

    response = requests.post("http://localhost:11434/api/chat", json=data)
    result = response.json()

    # извлекаем текстовый ответ
    text = result["message"]["content"]

    # пытаемся распарсить JSON
    import json
    try:
        parsed = json.loads(text)
    except json.JSONDecodeError:
        parsed = {
            "plant_class": "invalid_json",
            "disease_class": "invalid_json",
            "plant_confidence": 0,
            "disease_confidence": 0,
            "explanation": text
        }

    return parsed


# -------- MAIN LOOP --------
def main():
    # создаём CSV
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            "image_name",
            "plant_class",
            "disease_class",
            "plant_confidence",
            "disease_confidence",
            "explanation",
            "processing_time_sec"
        ])

        # перебор всех файлов
        for filename in os.listdir(DATASET_DIR):
            if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            image_path = os.path.join(DATASET_DIR, filename)
            print(f"Processing {filename}...")

            start = time.time()

            # кодирование + классификация
            b64 = encode_image(image_path)
            result = classify_image(b64)

            elapsed = round(time.time() - start, 3)

            # запись в файл
            writer.writerow([
                filename,
                result.get("plant_class"),
                result.get("disease_class"),
                result.get("plant_confidence"),
                result.get("disease_confidence"),
                result.get("explanation"),
                elapsed
            ])

            print(f" → done in {elapsed} sec")

    print("\n✨ Finished! Results saved to:", OUTPUT_CSV)


if __name__ == "__main__":
    main()
